In [8]:
using Rocket
using Distributions
using ReactiveMP
using BenchmarkTools

using Profile
using PProf
using ProfileSVG

import Base: show

In [9]:
function createSubgraph(index::Int)
    noise_add = AdditionNode()
    
    noise = constvar(:noise, Normal(0.0, sqrt(200.0)))
    yn    = datavar(:yn, Float64)
    
    connect!(noise_add, :in2, noise, 1)
    connect!(noise_add, :out, yn, 1)
    
    return (yn, noise_add)
end

function createGraph(size::Int)    
    c0_add = AdditionNode()
    
    c0       = constvar(:c0, 1.0)
    x0_prior = datavar(:x0_prior, Normal{Float64})
    
    connect!(c0_add, :in1, x0_prior, 1)
    connect!(c0_add, :in2, c0, 1)
    
    index = 1
    
    prev_c_add = c0_add
    
    xs = Vector{RandomVariable}(undef, size)
    ys = Vector{DataVariable}(undef, size)
    
    while index < size
        yn, noise_add = createSubgraph(index)
        xn            = randomvar(:xn, 3)
        
        c_add = AdditionNode()
        cn    = constvar(:cn, 1.0)
        
        connect!(prev_c_add, :out, xn, 1)
        connect!(noise_add, :in1, xn, 2)
        connect!(c_add, :in1, xn, 3)
        connect!(c_add, :in2, cn, 1)
        
        activate!(prev_c_add)
        activate!(noise_add)
        
        xs[index] = xn
        ys[index] = yn
        
        prev_c_add = c_add
        
        index += 1
    end
    
    last_noise_add = AdditionNode();
    
    x_last     = randomvar(:x_last, 2)
    y_last     = datavar(:ylast, Float64)
    last_noise = constvar(:last_noise, Normal(0.0, sqrt(200.0)))
    
    connect!(prev_c_add, :out, x_last, 1)
    connect!(last_noise_add, :in1, x_last, 2)
    connect!(last_noise_add, :in2, last_noise, 1)
    connect!(last_noise_add, :out, y_last, 1)
    
    activate!(prev_c_add)
    activate!(last_noise_add)
    
    xs[size] = x_last
    ys[size] = y_last
    
    return (xs, ys, x0_prior)
end

createGraph (generic function with 1 method)

In [10]:
function smoothing(data)
    N = length(data)
    
    xs, ys, x_prior = createGraph(N);
    
    messages      = Vector{Normal{Float64}}(undef, N)
    subscriptions = Vector{Teardown}(undef, N)
    
    @inbounds for (index, x) in enumerate(xs)
        subscriptions[index] = subscribe!(getmarginal(x), (d) -> messages[index] = getdata(d))
    end
    
    update!(x_prior, Normal(0.0, sqrt(10000.0)))
    for i in 1:N
       update!(ys[i], data[i])
    end
    
    foreach(unsubscribe!, subscriptions)
    
    return messages
end

smoothing (generic function with 1 method)

In [11]:
N = 600
data = collect(1:N) + sqrt(200.0) * randn(N);

In [14]:
@btime smoothing($data)

  59.308 ms (417573 allocations: 15.62 MiB)


600-element Array{Normal{Float64},1}:
 Normal{Float64}(μ=1.5312576947689023, σ=0.5773406469256954)
 Normal{Float64}(μ=2.5312576947689025, σ=0.5773406469256954)
 Normal{Float64}(μ=3.5312576947689025, σ=0.5773406469256954)
 Normal{Float64}(μ=4.531257694768902, σ=0.5773406469256953)
 Normal{Float64}(μ=5.531257694768903, σ=0.5773406469256954)
 Normal{Float64}(μ=6.531257694768904, σ=0.5773406469256954)
 Normal{Float64}(μ=7.531257694768905, σ=0.5773406469256954)
 Normal{Float64}(μ=8.531257694768906, σ=0.5773406469256954)
 Normal{Float64}(μ=9.531257694768904, σ=0.5773406469256954)
 Normal{Float64}(μ=10.531257694768904, σ=0.5773406469256953)
 Normal{Float64}(μ=11.531257694768904, σ=0.5773406469256954)
 Normal{Float64}(μ=12.531257694768902, σ=0.5773406469256953)
 Normal{Float64}(μ=13.5312576947689, σ=0.5773406469256954)
 ⋮
 Normal{Float64}(μ=589.5312576947679, σ=0.577340646925696)
 Normal{Float64}(μ=590.5312576947679, σ=0.577340646925696)
 Normal{Float64}(μ=591.5312576947679, σ=0.57734064692569

In [13]:
@time smoothing(data)

  0.069567 seconds (417.57 k allocations: 15.622 MiB)


600-element Array{Normal{Float64},1}:
 Normal{Float64}(μ=1.5312576947689023, σ=0.5773406469256954)
 Normal{Float64}(μ=2.5312576947689025, σ=0.5773406469256954)
 Normal{Float64}(μ=3.5312576947689025, σ=0.5773406469256954)
 Normal{Float64}(μ=4.531257694768902, σ=0.5773406469256953)
 Normal{Float64}(μ=5.531257694768903, σ=0.5773406469256954)
 Normal{Float64}(μ=6.531257694768904, σ=0.5773406469256954)
 Normal{Float64}(μ=7.531257694768905, σ=0.5773406469256954)
 Normal{Float64}(μ=8.531257694768906, σ=0.5773406469256954)
 Normal{Float64}(μ=9.531257694768904, σ=0.5773406469256954)
 Normal{Float64}(μ=10.531257694768904, σ=0.5773406469256953)
 Normal{Float64}(μ=11.531257694768904, σ=0.5773406469256954)
 Normal{Float64}(μ=12.531257694768902, σ=0.5773406469256953)
 Normal{Float64}(μ=13.5312576947689, σ=0.5773406469256954)
 ⋮
 Normal{Float64}(μ=589.5312576947679, σ=0.577340646925696)
 Normal{Float64}(μ=590.5312576947679, σ=0.577340646925696)
 Normal{Float64}(μ=591.5312576947679, σ=0.57734064692569

In [12]:
@time smoothing(data)

  0.739958 seconds (1.74 M allocations: 81.098 MiB, 4.24% gc time)


600-element Array{Normal{Float64},1}:
 Normal{Float64}(μ=1.5312576947689023, σ=0.5773406469256954)
 Normal{Float64}(μ=2.5312576947689025, σ=0.5773406469256954)
 Normal{Float64}(μ=3.5312576947689025, σ=0.5773406469256954)
 Normal{Float64}(μ=4.531257694768902, σ=0.5773406469256953)
 Normal{Float64}(μ=5.531257694768903, σ=0.5773406469256954)
 Normal{Float64}(μ=6.531257694768904, σ=0.5773406469256954)
 Normal{Float64}(μ=7.531257694768905, σ=0.5773406469256954)
 Normal{Float64}(μ=8.531257694768906, σ=0.5773406469256954)
 Normal{Float64}(μ=9.531257694768904, σ=0.5773406469256954)
 Normal{Float64}(μ=10.531257694768904, σ=0.5773406469256953)
 Normal{Float64}(μ=11.531257694768904, σ=0.5773406469256954)
 Normal{Float64}(μ=12.531257694768902, σ=0.5773406469256953)
 Normal{Float64}(μ=13.5312576947689, σ=0.5773406469256954)
 ⋮
 Normal{Float64}(μ=589.5312576947679, σ=0.577340646925696)
 Normal{Float64}(μ=590.5312576947679, σ=0.577340646925696)
 Normal{Float64}(μ=591.5312576947679, σ=0.57734064692569